In [12]:
from glob import glob
from upaths import file_paths,tiles_dpath,patches_dpath

from geotile import GeoTile
from os.path import join
from os import makedirs,system
from concurrent.futures import ProcessPoolExecutor

def geotile_generate_tiles(tif_path, dir_path):
   
   gt = GeoTile(tif_path)
   gt.generate_tiles(dir_path,tile_x=256, tile_y=256, stride_x=256, stride_y=256)
   gt.close()


def tileindex(tdir,gpkg): #pkg = tdir + '.gpkg'
   cmd = f'gdaltindex -t_srs EPSG:4326 -f GPKG {gpkg} {join(tdir)}/*.tif '
   system(cmd)

In [10]:
tilenames = [path.split('/')[-1].replace('_tdem_DEM.tif', '') for path in file_paths]
print(tilenames)

['N09E105', 'N09E106', 'N10E104', 'N10E105', 'N10E106', 'N11E104', 'N11E105', 'N12E103', 'N12E104', 'N12E105', 'N13E103', 'N13E104', 'N13E105', 'S01W063', 'S01W064', 'S02W063', 'S02W064']


gen patches tif 

In [11]:
with ProcessPoolExecutor() as PEX:
    for file_path, tile_name in zip(file_paths, tilenames):
        #print(file_path,'',tile_name)
        tdir = join(tiles_dpath, tile_name)
        makedirs(tdir, exist_ok=True)
        #PEX.submit(geotile_generate_tiles, file_path, tdir)
                


gen patches gpkg

In [ ]:
with ProcessPoolExecutor() as PEX:
    for tile_name in  tilenames:
        tdir = join(tiles_dpath, tile_name)
        print(tdir)
        tfiles = glob(f'{tdir}/*.tif')
        print(len(tfiles))
        print(tdir,'',tile_name)
        #pdir = join(patches_dpath, tile_name)
        gpkg = join(patches_dpath,f'{tile_name}.gpkg' )
        print(gpkg)
        #makedirs(pdir, exist_ok=True)
        PEX.submit(tileindex,tdir,gpkg)